In [6]:
import boto3
import json
print('Running boto3 version:', boto3.__version__)

Running boto3 version: 1.34.138


# Configuration

In [7]:
boto3_session = boto3.session.Session()
bedrock = boto3.client(service_name='bedrock-runtime')

# Variables

In [8]:
model_id = "meta.llama3-70b-instruct-v1:0"

In [9]:
init_configuration = """You are an anonimization tool in identifying attributes in texts that can identify or quasi-identify a user."""

In [10]:
orden = """
Devuelve solo el texto original con la identificación y etiquetación del texto con la información personal del paciente añadiendolo entre claudators, por ejemplo: [**arquitecto**], [**8/9/21**], [**padre**]. No comentes nada más
"""
#Las etiquetas posibles que puedes usar son [**NUMERO_TELEFONO**], [**FECHAS**], [**FAMILIARES_SUJETO_ASISTENCIA**], [**PROFESION**] y [**SÍNTOMAS**].  No puedes inventarte otras etiquetas.

In [11]:
with open('./data/processed/txt/CARMEN-I_CC_1.txt', 'r') as archivo:
    text = archivo.read()
    print(text)

Primera llamada HDOM COVID-19 POSITIVO
.
Trabaja en hilandero
Vive con suegro y 2 yernos, empezaron con sintomas digestius previos Ant med: rinitis alergica
Tto: no
Alergias: niega
.
Fecha frotis SARS-CoV-2: 8/9/21 por control rutinario Fecha inicio síntomas: 7/9/21 amb sx digestius...
.
- Fiebre: no
- Disnea: no
- Tos: no
- Diarrea: deposiciones mes frequents
- Otros: no
.
PLAN:
Se explican medidas de aislamiento.
Tomar temperatura dos veces al día. Doy teléfonos de contacto (Teléfono HDOM, de lunes a domingo de 8h a 20h: 976028134 / fuera de este horario deberán llamar a la centralita del hospital: 983425634 y pedir que les comuniquen con el busca de infecciosas).Llamada de control COVID
Fecha frotis SARS-CoV-2: 8/9/21 por control rutinario Fecha inicio síntomas: 7/9/21 amb sx digestius...
.
Asintomatic.
Anticossos negatius tant IgG com IgA.
PLAN
Trucar a final de setmana.Llamada de control COVID
Fecha frotis SARS-CoV-2: 8/9/21 por control rutinario Fecha inicio síntomas: 7/9/21 dige

In [12]:
prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
{init_configuration}
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
{orden}
{text}
<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

In [13]:
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are an anonimization tool in identifying attributes in texts that can identify or quasi-identify a user.
<|eot_id|>
<|start_header_id|>user<|end_header_id|>

Devuelve solo el texto original con la identificación y etiquetación del texto con la información personal del paciente añadiendolo entre claudators, por ejemplo: [**arquitecto**], [**8/9/21**], [**padre**]. No comentes nada más

Primera llamada HDOM COVID-19 POSITIVO
.
Trabaja en hilandero
Vive con suegro y 2 yernos, empezaron con sintomas digestius previos Ant med: rinitis alergica
Tto: no
Alergias: niega
.
Fecha frotis SARS-CoV-2: 8/9/21 por control rutinario Fecha inicio síntomas: 7/9/21 amb sx digestius...
.
- Fiebre: no
- Disnea: no
- Tos: no
- Diarrea: deposiciones mes frequents
- Otros: no
.
PLAN:
Se explican medidas de aislamiento.
Tomar temperatura dos veces al día. Doy teléfonos de contacto (Teléfono HDOM, de lunes a domingo de 8h a 20h: 976028134 / fuera d

In [14]:
max_gen_len = 1000
temperature = 0.1
top_p = 0.9

In [15]:
body = json.dumps({
    "prompt":prompt,
    "max_gen_len":max_gen_len,
    "temperature":temperature,
    "top_p":top_p
})

In [16]:
response = bedrock.invoke_model(body=body, modelId=model_id)

In [17]:
response_body = json.loads(response.get('body').read())

In [18]:
response_body['generation']

'\n\nPrimera llamada HDOM COVID-19 POSITIVO\n.\nTrabaja en [**hilandero**]\nVive con [**suegro**] y 2 [**yernos**], empezaron con sintomas digestius previos Ant med: [**rinitis alergica**]\nTto: no\nAlergias: niega\n.\nFecha frotis SARS-CoV-2: [**8/9/21**] por control rutinario Fecha inicio síntomas: [**7/9/21**] amb sx digestius...\n.\n- Fiebre: no\n- Disnea: no\n- Tos: no\n- Diarrea: deposiciones mes frequents\n- Otros: no\n.\nPLAN:\nSe explican medidas de aislamiento.\nTomar temperatura dos veces al día. Doy teléfonos de contacto (Teléfono HDOM, de lunes a domingo de 8h a 20h: [**976028134**] / fuera de este horario deberán llamar a la centralita del hospital: [**983425634**] y pedir que les comuniquen con el busca de infecciosas).Llamada de control COVID\nFecha frotis SARS-CoV-2: [**8/9/21**] por control rutinario Fecha inicio síntomas: [**7/9/21**] amb sx digestius...\n.\nAsintomatic.\nAnticossos negatius tant IgG com IgA.\nPLAN\nTrucar a final de setmana.Llamada de control COVID\

In [19]:
print(f"Generated Text: {response_body['generation']}")
print(f"Prompt Token count:  {response_body['prompt_token_count']}")
print(f"Generation Token count:  {response_body['generation_token_count']}")
print(f"Stop reason:  {response_body['stop_reason']}")

Generated Text: 

Primera llamada HDOM COVID-19 POSITIVO
.
Trabaja en [**hilandero**]
Vive con [**suegro**] y 2 [**yernos**], empezaron con sintomas digestius previos Ant med: [**rinitis alergica**]
Tto: no
Alergias: niega
.
Fecha frotis SARS-CoV-2: [**8/9/21**] por control rutinario Fecha inicio síntomas: [**7/9/21**] amb sx digestius...
.
- Fiebre: no
- Disnea: no
- Tos: no
- Diarrea: deposiciones mes frequents
- Otros: no
.
PLAN:
Se explican medidas de aislamiento.
Tomar temperatura dos veces al día. Doy teléfonos de contacto (Teléfono HDOM, de lunes a domingo de 8h a 20h: [**976028134**] / fuera de este horario deberán llamar a la centralita del hospital: [**983425634**] y pedir que les comuniquen con el busca de infecciosas).Llamada de control COVID
Fecha frotis SARS-CoV-2: [**8/9/21**] por control rutinario Fecha inicio síntomas: [**7/9/21**] amb sx digestius...
.
Asintomatic.
Anticossos negatius tant IgG com IgA.
PLAN
Trucar a final de setmana.Llamada de control COVID
Fecha frot

# Metricas

In [20]:
text_generated = response_body['generation']

In [21]:
print(text_generated)



Primera llamada HDOM COVID-19 POSITIVO
.
Trabaja en [**hilandero**]
Vive con [**suegro**] y 2 [**yernos**], empezaron con sintomas digestius previos Ant med: [**rinitis alergica**]
Tto: no
Alergias: niega
.
Fecha frotis SARS-CoV-2: [**8/9/21**] por control rutinario Fecha inicio síntomas: [**7/9/21**] amb sx digestius...
.
- Fiebre: no
- Disnea: no
- Tos: no
- Diarrea: deposiciones mes frequents
- Otros: no
.
PLAN:
Se explican medidas de aislamiento.
Tomar temperatura dos veces al día. Doy teléfonos de contacto (Teléfono HDOM, de lunes a domingo de 8h a 20h: [**976028134**] / fuera de este horario deberán llamar a la centralita del hospital: [**983425634**] y pedir que les comuniquen con el busca de infecciosas).Llamada de control COVID
Fecha frotis SARS-CoV-2: [**8/9/21**] por control rutinario Fecha inicio síntomas: [**7/9/21**] amb sx digestius...
.
Asintomatic.
Anticossos negatius tant IgG com IgA.
PLAN
Trucar a final de setmana.Llamada de control COVID
Fecha frotis SARS-CoV-2: [

In [403]:
import re
from collections import Counter

def calc_metrics(ground_truth, predictions):
    ground_truth_counter = Counter(ground_truth)
    predictions_counter = Counter(predictions)

    true_positives = sum((ground_truth_counter & predictions_counter).values())
    false_positives = sum((predictions_counter - ground_truth_counter).values())
    false_negatives = sum((ground_truth_counter - predictions_counter).values())

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1

def evaluate(masked, generated):
    """ 
    Input: 
        - masked (str): Ground_truth text
        - generated(str): Text to be evaluated

    Output:
        - Precision, Recall and F1 (float)
    """
    ground_truth = re.findall(r'\[\*\*(.*?)\*\*\]', masked)
    predictions = re.findall(r'\[\*\*(.*?)\*\*\]', generated)
    print(ground_truth)
    print(predictions)
    
    return calc_metrics(ground_truth, predictions)

In [26]:
with open('./data/processed/masked/CARMEN-I_CC_1.txt', 'r') as archivo:
    text_hoped = archivo.read()
    print(text_hoped)

Primera llamada HDOM COVID-19 POSITIVO
.
Trabaja en [**hilandero**]
Vive con [**suegro**] y [**2 yernos**], empezaron con sintomas digestius previos Ant med: rinitis alergica
Tto: no
Alergias: niega
.
Fecha frotis SARS-CoV-2: [**8/9/21**] por control rutinario Fecha inicio síntomas: [**7/9/21**] amb sx digestius...
.
- Fiebre: no
- Disnea: no
- Tos: no
- Diarrea: deposiciones mes frequents
- Otros: no
.
PLAN:
Se explican medidas de aislamiento.
Tomar temperatura dos veces al día. Doy teléfonos de contacto (Teléfono HDOM, de lunes a domingo de 8h a 20h: [**976028134**] / fuera de este horario deberán llamar a la centralita del hospital: [**983425634**] y pedir que les comuniquen con el busca de infecciosas).Llamada de control COVID
Fecha frotis SARS-CoV-2: [**8/9/21**] por control rutinario Fecha inicio síntomas: [**7/9/21**] amb sx digestius...
.
Asintomatic.
Anticossos negatius tant IgG com IgA.
PLAN
Trucar a final de setmana.Llamada de control COVID
Fecha frotis SARS-CoV-2: [**8/9/21

In [27]:
cal_met = evaluate(text_hoped, text_generated)
print(f"Precision: {cal_met[0]}")
print(f"Recall: {cal_met[1]}")
print(f"F1: {cal_met[2]}")

['hilandero', 'suegro', '2 yernos', '8/9/21', '7/9/21', '976028134', '983425634', '8/9/21', '7/9/21', '8/9/21', '7/9/21', '8/9/21', '7/9/21', 'Salud laboral', '15/09', '10.09.2021']
['hilandero', 'suegro', 'yernos', 'rinitis alergica', '8/9/21', '7/9/21', '976028134', '983425634', '8/9/21', '7/9/21', '8/9/21', '7/9/21', '15/09', '8/9/21', '7/9/21', '10.09.2021']
Precision: 0.8888888888888888
Recall: 0.8
F1: 0.8421052631578948


## Cosine similarity

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform([text_hoped, text_generated])

cosine_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])

print(f"La similitud del coseno entre los textos es: {cosine_sim[0][0]}")

La similitud del coseno entre los textos es: 0.9910432368486771


## Levenshtein Distance

In [364]:
from tqdm import tqdm

def levenshtein_distance(s1, s2, show_progress=True):
    """
    Calcula la distancia de Levenshtein entre dos cadenas.

    La distancia de Levenshtein es el número mínimo de operaciones de edición 
    (inserción, eliminación o sustitución de un carácter) necesarias para 
    transformar una cadena en otra.

    Parámetros:
        s1 (str): Primera cadena
        s2 (str): Segunda cadena
        show_progress (bool): Si es True, muestra una barra de progreso. 
                              Por defecto es False.
    Retorna:
        int: La distancia de Levenshtein entre s1 y s2
    """
    # Usar tqdm solo si show_progress es True
    iterable = tqdm(s1) if show_progress else s1

    if len(s1) < len(s2):
        s1, s2 = s2, s1
    if len(s2) == 0:
        return len(s1)

    previous_row = list(range(len(s2) + 1))
    for i, c1 in enumerate(iterable):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    return previous_row[-1]



In [401]:
[text_generated, text_gt] = ["""Primera llamada HDOM COVID-19 POSITIVO
.
Trabaja en [**hilandero**]
Vive con [**suegro**] y 2 [**yernos**], empezaron con sintomas digestius previos Ant med: [**rinitis alergica**]
Tto: no
Alergias: niega
.
Fecha frotis SARS-CoV-2: [**8/9/21**] por control rutinario Fecha inicio síntomas: [**7/9/21**] amb sx digestius...
.
- Fiebre: no
- Disnea: no
- Tos: no
- Diarrea: deposiciones mes frequents
- Otros: no
.
PLAN:
Se explican medidas de aislamiento.
Tomar temperatura dos veces al día. Doy teléfonos de contacto (Teléfono HDOM, de lunes a domingo de 8h a 20h: [**976028134**] / fuera de este horario deberán llamar a la centralita del hospital: [**983425634**] y pedir que les comuniquen con el busca de infecciosas).Llamada de control COVID
Fecha frotis SARS-CoV-2: [**8/9/21**] por control rutinario Fecha inicio síntomas: [**7/9/21**] amb sx digestius...
.
Asintomatic.
Anticossos negatius tant IgG com IgA.
PLAN
Trucar a final de setmana.Llamada de control COVID
Fecha frotis SARS-CoV-2: [**8/9/21**] por control rutinario Fecha inicio síntomas: [**7/9/21**] digestivos
.
Asintomatico.
Anticuerpos negativos, IgG e IgA
PLAN
Llamar [**15/09**]Llamada de control COVID
Fecha frotis SARS-CoV-2: [**8/9/21**] por control rutinario Fecha inicio síntomas: [**7/9/21**] digestivos
.
Anticuerpos negativos [**10.09.2021**], IgG e IgA
.
PLAN: alta HDOM, pendiente frotis por parte de Salud laboral.""", """Primera llamada HDOM COVID-19 POSITIVO
.
Trabaja en [**hilandero**]
Vive con [**suegro**] y [**2 yernos**], empezaron con sintomas digestius previos Ant med: rinitis alergica
Tto: no
Alergias: niega
.
Fecha frotis SARS-CoV-2: [**8/9/21**] por control rutinario Fecha inicio síntomas: [**7/9/21**] amb sx digestius...
.
- Fiebre: no
- Disnea: no
- Tos: no
- Diarrea: deposiciones mes frequents
- Otros: no
.
PLAN:
Se explican medidas de aislamiento.
Tomar temperatura dos veces al día. Doy teléfonos de contacto (Teléfono HDOM, de lunes a domingo de 8h a 20h: [**976028134**] / fuera de este horario deberán llamar a la centralita del hospital: [**983425634**] y pedir que les comuniquen con el busca de infecciosas).Llamada de control COVID
Fecha frotis SARS-CoV-2: [**8/9/21**] por control rutinario Fecha inicio síntomas: [**7/9/21**] amb sx digestius...
.
Asintomatic.
Anticossos negatius tant IgG com IgA.
PLAN
Trucar a final de setmana.Llamada de control COVID
Fecha frotis SARS-CoV-2: [**8/9/21**] por control rutinario Fecha inicio síntomas: [**7/9/21**] digestivos
.
Asintomatico.
Anticuerpos negativos, IgG e IgA
PLAN
Llamar 15/09Llamada de control COVID
Fecha frotis SARS-CoV-2: [**8/9/21**] por control rutinario Fecha inicio síntomas: [**7/9/21**] digestivos
.
Anticuerpos negativos 10.09.2021, IgG e IgA
.
PLAN: alta HDOM, pendiente frotis por parte de [**Salud laboral**][**15/09**]Llamada de control COVID
Fecha frotis SARS-CoV-2: 8/9/21 por control rutinario Fecha inicio síntomas: 7/9/21 digestivos
.
Anticuerpos negativos [**10.09.2021**], IgG e IgA
.
PLAN: alta HDOM, pendiente frotis por parte de Salud laboral."""]

In [402]:
text_generated = text_generated.replace('[**', '').replace('**]', '')
text_gt = text_gt.replace('[**', '').replace('**]', '')

result = levenshtein_distance(text_generated, text_gt[:len(text_generated)], show_progress=False)
print(f'Levenshtein Distance: {result}')

Levenshtein Distance: 1
